In [5]:
import pandas as pd
import numpy as np

In [9]:
gold_std_filepaths = [
    "/Users/rushil/brain_extraction/results/quantitative/gold_std_comparison/JH_Head_CT/method_vs_gold_per_scan_JH_Head_CT.csv",
    "/Users/rushil/brain_extraction/results/quantitative/gold_std_comparison/CERMEP-IDB-MRXFDG/method_vs_gold_per_scan_CERMEP-IDB-MRXFDG.csv",
    "/Users/rushil/brain_extraction/results/quantitative/gold_std_comparison/MISTIE_III/method_vs_gold_per_scan_mistie3.csv",
]

all_df = pd.DataFrame()
for path in gold_std_filepaths:
    df = pd.read_csv(path)
    df['Dataset'] = path.split('/')[-2]
    df.drop(columns=['gold_path', 'pred_path'], inplace=True, errors='ignore')
    
    # Filter out unwanted stems
    if 'stem' in df.columns:
        df = df[~df['stem'].str.contains('_BRAIN_2_Anonymized', na=False)]
        df = df[df['stem'] != '13_BONE_1_Anonymized']
    
    all_df = pd.concat([all_df, df], ignore_index=True)

all_df['icv_ratio'] = all_df['icv_ml_pred'] / all_df['icv_ml_ref']
display(all_df.head())
print(len(all_df.stem.unique()))

,patient_id,stem,method_A,method_B,dice,iou,sensitivity,specificity,msd_mm,hd95_mm,icv_ml_ref,icv_ml_pred,delta_icv_ml,above_thresh,within_5ml,Dataset,icv_ratio
0,1,01_BRAIN_1_Anonymized,Brainchop,GoldManual,0.960,0.922,0.986,0.989,1.399,3.207,1217.472,1284.200,66.727,0,0,JH_Head_CT,1.054809
1,1,01_BRAIN_1_Anonymized,HD-CTBET,GoldManual,0.939,0.886,0.985,0.981,2.486,5.000,1217.472,1335.502,118.030,0,0,JH_Head_CT,1.096947
2,1,01_BRAIN_1_Anonymized,SynthStrip,GoldManual,0.942,0.890,0.991,0.981,2.211,3.784,1217.472,1346.028,128.556,0,0,JH_Head_CT,1.105593
3,1,01_BRAIN_1_Anonymized,CTBET,GoldManual,0.991,0.983,0.992,0.998,0.223,0.635,1217.472,1220.469,2.997,1,1,JH_Head_CT,1.002462
4,1,01_BRAIN_1_Anonymized,Robust-CTBET,GoldManual,0.989,0.978,0.987,0.998,0.427,1.004,1217.472,1213.580,-3.893,1,1,JH_Head_CT,0.996803


65


In [10]:
import plotly.graph_objects as go
import plotly.express as px

datasets = sorted(all_df['Dataset'].unique())
methods  = sorted(all_df['method_A'].unique())

# Custom dark, color-blind friendly colors per dataset
# Explicit mapping for reproducibility; extend if new datasets appear.
custom_dataset_colors = {
    'JH_Head_CT': '#264653',        # dark teal
    'CERMEP-IDB-MRXFDG': '#6a4c93', # dark purple
    'MISTIE_III': '#b23a48',        # deep wine red
}
# Fallback palette (Dark2) for any dataset not in the explicit map
fallback_colors = px.colors.qualitative.Dark2
for i, ds in enumerate(datasets):
    if ds not in custom_dataset_colors:
        custom_dataset_colors[ds] = fallback_colors[i % len(fallback_colors)]

fig = go.Figure()

# One trace per dataset; Plotly groups by x (method)
for ds in datasets:
    d = all_df[all_df['Dataset'] == ds]
    if d.empty:
        continue
    fig.add_trace(go.Box(
        x=d['method_A'],                # categories = methods
        y=d['dice'],
        name=ds,                        # legend per dataset
        marker_color=custom_dataset_colors[ds],
        boxpoints='outliers',
        jitter=0,
        pointpos=0,
        legendgroup=ds,
        offsetgroup=ds,                 # group within each method
        alignmentgroup='all',
        hovertemplate='Dataset: '+ds+'<br>Method: %{x}<br>Dice: %{y:.3f}<extra></extra>'
    ))

fig.update_layout(
    boxmode='group',
    width=1100, height=600,
    margin=dict(l=60, r=30, t=60, b=50),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='left', x=0, title=None),
    font=dict(size=13)
)
fig.update_xaxes(title_text='Method',
                 categoryorder='array', categoryarray=methods, tickmode='array',
                 tickvals=methods, ticktext=methods)
fig.update_yaxes(title_text='Dice')

fig.show()

In [11]:
import plotly.graph_objects as go
import plotly.express as px

datasets = sorted(all_df['Dataset'].unique())
methods  = sorted(all_df['method_A'].unique())

# Custom dark, color-blind friendly colors per dataset
# Explicit mapping for reproducibility; extend if new datasets appear.
custom_dataset_colors = {
    'JH_Head_CT': '#264653',        # dark teal
    'CERMEP-IDB-MRXFDG': '#6a4c93', # dark purple
    'MISTIE_III': '#b23a48',        # deep wine red
}
# Fallback palette (Dark2) for any dataset not in the explicit map
fallback_colors = px.colors.qualitative.Dark2
for i, ds in enumerate(datasets):
    if ds not in custom_dataset_colors:
        custom_dataset_colors[ds] = fallback_colors[i % len(fallback_colors)]

fig = go.Figure()

# One trace per dataset; Plotly groups by x (method)
for ds in datasets:
    d = all_df[all_df['Dataset'] == ds]
    if d.empty:
        continue
    fig.add_trace(go.Box(
        x=d['method_A'],                # categories = methods
        y=d['iou'],
        name=ds,                        # legend per dataset
        marker_color=custom_dataset_colors[ds],
        boxpoints='outliers',
        jitter=0,
        pointpos=0,
        legendgroup=ds,
        offsetgroup=ds,                 # group within each method
        alignmentgroup='all',
        hovertemplate='Dataset: '+ds+'<br>Method: %{x}<br>IoU: %{y:.3f}<extra></extra>'
    ))

fig.update_layout(
    boxmode='group',
    width=1100, height=600,
    margin=dict(l=60, r=30, t=60, b=50),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='left', x=0, title=None),
    font=dict(size=13)
)
fig.update_xaxes(title_text='Method',
                 categoryorder='array', categoryarray=methods, tickmode='array',
                 tickvals=methods, ticktext=methods)
fig.update_yaxes(title_text='IoU')

fig.show()

In [13]:
import plotly.graph_objects as go
import plotly.express as px

datasets = sorted(all_df['Dataset'].unique())
methods  = sorted(all_df['method_A'].unique())

# Custom dark, color-blind friendly colors per dataset
# Explicit mapping for reproducibility; extend if new datasets appear.
custom_dataset_colors = {
    'JH_Head_CT': '#264653',        # dark teal
    'CERMEP-IDB-MRXFDG': '#6a4c93', # dark purple
    'MISTIE_III': '#b23a48',        # deep wine red
}
# Fallback palette (Dark2) for any dataset not in the explicit map
fallback_colors = px.colors.qualitative.Dark2
for i, ds in enumerate(datasets):
    if ds not in custom_dataset_colors:
        custom_dataset_colors[ds] = fallback_colors[i % len(fallback_colors)]

fig = go.Figure()

# One trace per dataset; Plotly groups by x (method)
for ds in datasets:
    d = all_df[all_df['Dataset'] == ds]
    if d.empty:
        continue
    fig.add_trace(go.Box(
        x=d['method_A'],                # categories = methods
        y=d['icv_ratio'],
        name=ds,                        # legend per dataset
        marker_color=custom_dataset_colors[ds],
        boxpoints='outliers',
        jitter=0,
        pointpos=0,
        legendgroup=ds,
        offsetgroup=ds,                 # group within each method
        alignmentgroup='all',
        hovertemplate='Dataset: '+ds+'<br>Method: %{x}<br>ICV Ratio: %{y:.3f}<extra></extra>'
    ))

fig.update_layout(
    boxmode='group',
    width=1100, height=600,
    margin=dict(l=60, r=30, t=60, b=50),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='left', x=0, title=None),
    font=dict(size=13)
)
fig.update_xaxes(title_text='Method',
                 categoryorder='array', categoryarray=methods, tickmode='array',
                 tickvals=methods, ticktext=methods)
fig.update_yaxes(title_text='ICV Ratio')

fig.show()

In [21]:
import plotly.graph_objects as go
import plotly.express as px

datasets = sorted(all_df['Dataset'].unique())
methods  = sorted(all_df['method_A'].unique())

# Custom dark, color-blind friendly colors per dataset
# Explicit mapping for reproducibility; extend if new datasets appear.
custom_dataset_colors = {
    'JH_Head_CT': '#264653',        # dark teal
    'CERMEP-IDB-MRXFDG': '#6a4c93', # dark purple
    'MISTIE_III': '#b23a48',        # deep wine red
}
# Fallback palette (Dark2) for any dataset not in the explicit map
fallback_colors = px.colors.qualitative.Dark2
for i, ds in enumerate(datasets):
    if ds not in custom_dataset_colors:
        custom_dataset_colors[ds] = fallback_colors[i % len(fallback_colors)]

fig = go.Figure()

# One trace per dataset; Plotly groups by x (method)
for ds in datasets:
    d = all_df[all_df['Dataset'] == ds]
    if d.empty:
        continue
    fig.add_trace(go.Box(
        x=d['method_A'],                # categories = methods
        y=d['hd95_mm'],
        name=ds,                        # legend per dataset
        marker_color=custom_dataset_colors[ds],
        boxpoints='outliers',
        jitter=0,
        pointpos=0,
        legendgroup=ds,
        offsetgroup=ds,                 # group within each method
        alignmentgroup='all',
        hovertemplate='Dataset: '+ds+'<br>Method: %{x}<br>Hausdorff Distance (HD95) (mm): %{y:.3f}<extra></extra>'
    ))

fig.update_layout(
    boxmode='group',
    width=1100, height=600,
    margin=dict(l=60, r=30, t=60, b=50),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='left', x=0, title=None),
    font=dict(size=13)
)
fig.update_xaxes(title_text='Method',
                 categoryorder='array', categoryarray=methods, tickmode='array',
                 tickvals=methods, ticktext=methods)
fig.update_yaxes(title_text='Hausdorff Distance (HD95) (mm)')

fig.show()

In [22]:
import plotly.graph_objects as go
import plotly.express as px

datasets = sorted(all_df['Dataset'].unique())
methods  = sorted(all_df['method_A'].unique())

# Custom dark, color-blind friendly colors per dataset
# Explicit mapping for reproducibility; extend if new datasets appear.
custom_dataset_colors = {
    'JH_Head_CT': '#264653',        # dark teal
    'CERMEP-IDB-MRXFDG': '#6a4c93', # dark purple
    'MISTIE_III': '#b23a48',        # deep wine red
}
# Fallback palette (Dark2) for any dataset not in the explicit map
fallback_colors = px.colors.qualitative.Dark2
for i, ds in enumerate(datasets):
    if ds not in custom_dataset_colors:
        custom_dataset_colors[ds] = fallback_colors[i % len(fallback_colors)]

fig = go.Figure()

# One trace per dataset; Plotly groups by x (method)
for ds in datasets:
    d = all_df[all_df['Dataset'] == ds]
    if d.empty:
        continue
    fig.add_trace(go.Box(
        x=d['method_A'],                # categories = methods
        y=d['msd_mm'],
        name=ds,                        # legend per dataset
        marker_color=custom_dataset_colors[ds],
        boxpoints='outliers',
        jitter=0,
        pointpos=0,
        legendgroup=ds,
        offsetgroup=ds,                 # group within each method
        alignmentgroup='all',
        hovertemplate='Dataset: '+ds+'<br>Method: %{x}<br>Mean Squared Distance (mm): %{y:.3f}<extra></extra>'
    ))

fig.update_layout(
    boxmode='group',
    width=1100, height=600,
    margin=dict(l=60, r=30, t=60, b=50),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='left', x=0, title=None),
    font=dict(size=13)
)
fig.update_xaxes(title_text='Method',
                 categoryorder='array', categoryarray=methods, tickmode='array',
                 tickvals=methods, ticktext=methods)
fig.update_yaxes(title_text='Mean Squared Distance (mm)')

fig.show()